In [52]:
import srt

srt_fn = 'data/sample-0.srt'
srt_fn = 'data/dream/dream.srt'
doc_fn = 'data/dream/dream.docx'


with open (srt_fn) as rf :
    buf = rf.read ()
    
srt_data = srt.parse (buf)

last_time = '0:00:00.000000'

for sub in srt_data :
    #print (sub)
    print (sub.index, sub.start, sub.end, sub.content)
    last_time = sub.end
    break

print (last_time)
srt_list = list (srt_data)

1 0:00:00 0:00:02.580000 I have the pleasure to present to you,
0:00:02.580000


In [65]:
NUM_PARAGRAPHS = 2

last_time = srt_list [-1].end.total_seconds ()
split_length = int (last_time / NUM_PARAGRAPHS + 2)
print ('half_time in seconds :', split_length)

half_index = 1
last_index = len (srt_list)

"""
for sub in srt_list :
    index = sub.index
    st = sub.start.total_seconds ()
    et = sub.end.total_seconds ()
    content = sub.content
    
    if st >= half_time : ### less than 30 seconds
        half_index = index
        print (half_index, et)
        break
###
"""

print (last_time)

half_time in seconds : 500
996.64


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download ('punkt')

In [54]:
check_index = half_index

### 시간 기준으로 전체 문서를 나눔
def getParts (srt_list, split_length) : ### length in seconds
    split_time = split_length
    text_lines = []
    start_time = -1
    
    for sub in srt_list :
        index   = sub.index
        content = sub.content
        st      = sub.start.total_seconds ()
        et      = sub.end.total_seconds ()
        
        if start_time == -1 :
            start_time = st

        if st < split_time :
            if content != 'CHEERING AND APPLAUSE' :
                text_lines.append (content)
            continue
        ###
        
        print ('###', index, st, content)
        text = ' '.join (text_lines)
        yield text, start_time
        
        text_lines = []
        split_time += split_length
        start_time = -1
    ###
    
    text = ' '.join (text_lines)
    yield text, start_time
###


seg_length = 600 ### 10 minutes
parts = []
parts_start = []

it_parts = getParts (srt_list, split_length)

for text, start_time in it_parts :
    if text == '' : break

    parts.append (text)
    parts_start.append (start_time)
    
    print ('words :', len (text.split (' ')))
    print ('characters :', len (text))

    sentences = sent_tokenize (text)
    #print (len (sentences))
    #print ('\n'.join (sentences))
    print ('='*80)
###



### split text into small paragraphs : around 10000 characters each
def splitToParagraphs (text, split_length=1000) :
    tlen = len (text) ### total number of characters
    mods = int (tlen / split_length + 0.5) ### number of paragraphs
    para_size = int (tlen / mods) + 1 ### characters per paragraph
    print (tlen, mods, para_size)
    
    sentences = sent_tokenize (text)
    
    paragraphs = [] ### result
    total_len = 0
    buf = []
    
    for sentence in sentences :
        total_len += len (sentence)
        buf.append (sentence)
        
        #if total_len >= split_length :
        if total_len >= para_size :
            paragraph = ' '.join (buf)
            paragraphs.append (paragraph)
            
            total_len = 0
            buf = []
        ###
    ###

    if len (buf) > 0 :
        paragraph = ' '.join (buf)
        paragraphs.append (paragraph)

    return paragraphs
###
        

doc_text = []

for text in parts :
    paragraphs = splitToParagraphs (text)
    doc_text.append (paragraphs)
    
    for para in paragraphs :
        #print (para, '\n')
        pass
    ##
    
    print ('\n\n')
###

### 89 502.64 We cannot turn back.
words : 734
characters : 4013
words : 864
characters : 4591
4013 4 1004



4591 5 919





In [66]:
from docx import Document
from docx.shared import Pt
import time

# save to docx file
@staticmethod
def saveToDocx (title, doc_text, parts_start, time_length, filename) :
    document = Document()
    para = document.add_paragraph ()

    ### title
    run = para.add_run (title)
    run.font.size = Pt (16) # font size
    run.font.name = 'Arial' # font name
    run.bold = True

    ### body
    for idx, (paragraphs, start_time) in enumerate (zip (doc_text, parts_start)) :
        if idx == 0 :
            time_string = 'Length ' + time.strftime ('%H:%M:%S', time.gmtime (time_length))
        else :
            time_string = time.strftime ('%H:%M:%S', time.gmtime (start_time))
        
        body = document.add_paragraph ()
        run = body.add_run ('\n')
        run = body.add_run (time_string)
            
        run.font.size = Pt (14) # font size
        run.font.name = 'Arial' # font name
        run.bold = True

        for para_text in paragraphs :
            body = document.add_paragraph ()
            run = body.add_run ('\n')

            #sentences = text.split ('\n')
            run = body.add_run (para_text)
            run.font.size = Pt (14) # font size
            run.font.name = 'Arial' # font name

        run = body.add_run ('\n')
    ###
    
    ### save document
    document.save (filename)
###


#doc_file = output_dir + name + '.docx'
doc_file = doc_fn
title = '.'.join (doc_file.split ('/') [-1].split ('.') [:-1])

saveToDocx (title, doc_text, parts_start, last_time, doc_fn)
